In [18]:
from typing import Literal
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver # 持久化状态
from langgraph.graph import END, StateGraph, MessagesState # 状态图
from langgraph.prebuilt import ToolNode # 工具节点
import os

In [19]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

True

In [32]:
# 模拟一个天气查询工具
@tool # 将函数注册为LangGraph工具对象，否则无法被识别
def weather_search(query: str) -> str:
    '''天气查询工具'''
    if "广东" in query.lower() or "guangdong" in query.lower():
        return "现在广东的天气是多云，气温22度，湿度78%。"
    return "对不起，我无法获取该地区的天气信息。"

In [33]:
# 工具放入工具列表
tools = [weather_search]

# 创建工具结点
tool_node = ToolNode(tools)

# 初始划模型和工具
llm = ChatOpenAI(
    model="qwen3-next-80b-a3b-instruct",
    openai_api_key=os.getenv("QWEN_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.1
).bind_tools(tools) # LangGraph会让prompt里包含工具描述

In [22]:
# 决定是否继续执行
def should_continue(state: MessagesState) -> Literal["tools", END]:
    '''判断是否继续对话'''
    messages = state['messages']
    last_message = messages[-1] if messages else None
    # 如果最后一条消息是AI消息且包含工具调用，则继续使用工具
    if last_message.tool_calls:
        return "tools" # 路由到工具节点
    # 否则结束对话
    return END

In [23]:
def call_model(state: MessagesState):
    '''调用模型'''
    messages = state['messages']
    response = llm.invoke(messages)
    # 返回列表
    return {'messages': [response]}

In [28]:
# 定义状态图
workflow = StateGraph(MessagesState)
# 添加图结点
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 定义入口和边
workflow.set_entry_point("agent")

# 添加条件边
workflow.add_conditional_edges(
    "agent",
    should_continue
)

# 添加tools到agent的普通边
workflow.add_edge("tools", "agent")

# 初始化内存以在图运行期间保存状态
checkpointer = MemorySaver() # 可以存储到内存、文件、数据库等

# 编译图
# 这将编译成一个LangChain可运行对象，预检查
app = workflow.compile(checkpointer=checkpointer)

In [ ]:
# 运行图
fina_state = app.invoke(
    {
        "messages": [
            HumanMessage(content="请帮我查询一下广东的天气怎么样？")
        ]
    },
    config={"configurable":{"thread_id": 42}}
)

# 从最终状态中最后的消息获取结果
result = fina_state['messages'][-1].content
print(result)

广东现在的天气是多云，气温22度，湿度为78%。建议适当增减衣物，注意保持舒适。


In [30]:
fina_state = app.invoke(
    {
        "messages": [
            HumanMessage(content="我刚刚问的是哪个省的天气？")
        ]
    },
    config={"configurable":{"thread_id": 42}}
)

result = fina_state['messages'][-1].content
print(result)

您刚刚问的是广东省的天气。


In [31]:
graph_png = app.get_graph().draw_mermaid_png()
with open("langgraph.png", "wb") as f:
    f.write(graph_png)